# Communication

In [5]:
from ipaddress import IPv4Address 
from pyairmore.request import AirmoreSession  
from pyairmore.services.messaging import MessagingService  
c = "" 
ip = IPv4Address(c) 
s = AirmoreSession(ip)
print("Running:", s.is_server_running)
wa = s.request_authorization()
print("Authorization:",wa)
service = MessagingService(s) 

Running: True
Authorization: True


# Importing Libraries

In [6]:
import numpy
import numpy as npy 
import matplotlib.pyplot as plt
import os
import time
import cv2
# import tensorflow as tf

# YOLO

In [7]:
def main1(n1): 
    n = n1 
    global ld # Global variable for showing last detected time
    
    # load the COCO class labels our YOLO model was trained on - *preset
    lpath = os.path.sep.join(['yolo-coco', "coco.names"])
    la = open(lpath).read().strip().split("\n")
    # derive the paths to the YOLO weights and model configuration - *preset
    weightsPath = os.path.sep.join(['yolo-coco', "yolov3.weights"])
    configPath = os.path.sep.join(['yolo-coco', "yolov3.cfg"])
    # load our YOLO object detector trained on COCO dataset (80 classes) - *preset
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    
    # load input to get its dimensions
    im = cv2.imread(n)
    (H, W) = im.shape[:2]

    # Colour for the labels
    npy.random.seed(42)
    colours = npy.random.randint(0, 255, size=(len(la), 3),dtype="uint8")

    # Naming layers - *preset
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward - *preset
    # pass of the YOLO object detector, giving us our bounding boxes and associated probabilities
    blob = cv2.dnn.blobFromImage(im, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    out = net.forward(ln)
    end = time.time()

    box1 = []
    classID1 = []
    confidence1 = []
    
    for o in out:
        for det in o:
            s1 = det[5:]
            classID = npy.argmax(s1)
            confidence = s1[classID]
            if confidence > 0.5:
                box = det[0:4] * npy.array([W, H, W, H])
                (cX, cY, w1, h1) = box.astype("int")
                x = int(cX - (w1 / 2))
                y = int(cY - (h1 / 2))
                box1.append([x, y, int(w1), int(h1)])
                confidence1.append(float(confidence))
                classID1.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding - *preset
    id1 = cv2.dnn.NMSBoxes(box1, confidence1, 0.5, 0.3)
   
    if len(id1) > 0:
        for i in id1.flatten():
            temp = []
            (x, y) = (box1[i][0], box1[i][1])
            (w, h) = (box1[i][2], box1[i][3])
            cl = [int(c) for c in colours[classID1[i]]]
            
            text = "{}".format(la[classID1[i]])
            if text == "person":
                ct=time.time()
                cv2.rectangle(im, (x, y), (x + w, y + h), cl, 2)
                cv2.putText(im, str(i)+", "+str(x)+" "+(str(y)), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, cl, 2)
                if ct-ld>5:
                    service.send_message("9449277201", "Ambulance Detected !! Change Signal !") 
                    # print("Ambulance Detected !! Change Signal !")
                ld=time.time()
                cv2.imwrite("test/detected/time{}.jpg".format(ct),im)

# Main module

In [11]:
print("Enter video location")
vid1 = input()
# count = []
# count.append(0)
frames = 15 
# cv2.VideoCapture(0) - If you want webcam
cap = cv2.VideoCapture(vid1)
i,j,ld = 0,0,0

while True:
    r, f = cap.read() # takes the fram
    if r: 
        cv2.imshow('frame', f)   
        f = cv2.resize(f,(400,300))
        if i%frames == 0:
            try: 
                j = j+1
                s = "test/ss{}.jpg".format(j)
                cv2.imwrite(s,f)
                main1(s)
            except:
                pass
        i=i+1
        if cv2.waitKey(1) & 0xFF == ord('q'): 
        # Press Q to quit
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

Enter video location
Demo/Allen4.mp4
